In [5]:
import requests as rq
from bs4 import BeautifulSoup as bs
from collections import OrderedDict as od
#from threading import Thread
import threading
#from queuelib import queue
from multiprocessing import Queue
from selenium import webdriver
from datetime import datetime
import time
import random
import json
import csv
import traceback as tb
import re

In [2]:
#NUM_THREADS = 8

In [6]:
HOST = 'https://www.famicloud.com.tw'

In [7]:
index_url = HOST

In [18]:
if __name__ == '__main__': 
    q = Queue()  # 開一個 Queue 物件
    #打開一個名字叫 t1 的線程物件
    
    #這個物件會去呼叫 job1
    try:
        #同時t1導入 q 跟 lock 做現成控制
        #t1 = threading.Thread(target = small_branch_crawler, name = NUM_THREADS, args = (q ,)) 
        t1 = threading.Thread(target = famicloud_crawler, args = (q ,)) 
        t1.start()  #啟動 t1 線程
        t1.join()  #在 t1線程結束前阻止程式繼續運行    
        #確認Queue是否為空，如果不是就用 q.get() 取出值
    except:
        print("thread error")
    #確認Queue是否為空，如果不是就用 q.get() 取出值
    while not q.empty():   
        #print(q.get())
        #small_branch_url = q.get()
        small_branch_crawler(q.get())
    #確認Queue是否為空，如果不是就用 q.get() 取出值
        #while not q.empty():   
        #print(q.get())     
    
        

限時好康
限量9折
限量79折
超值買一送一
即期品半價
鮮食家推薦
Top30商品
假日追劇必吃
百元內美食
火鍋超值組
享受早午餐
露營方便吃
生猛海鮮
魚類
蝦\蟹\貝類
花枝\透抽\章魚\軟絲
海菜類
生猛海鮮品牌總覽
天然有機
家香豬
天和鮮物
台灣土雞王桂丁雞
幸美生技
御牧牛
雲嶺鮮雞
淳鮮
台灣好漁
柴米鮮BUY
大漢酵素
中晏生機
肉品肉類
豬肉
牛肉
羊肉
雞\鴨\鵝
肉品肉類品牌總覽
調理即食
餃子\湯包
滷味\小菜\沙拉
鮮食丸子
罐頭
麵點\麵食
鍋物\湯品
家常料理
米點\米食
火鍋料
寵物食品
即時調理品牌總覽
點心冰品
中式點心
西式點心\下午茶
月餅禮盒
沖泡飲品\果汁
冰品
休閒零食
點心冰品果汁品牌總覽
地方美食
基隆\台北美食
桃園\新竹美食
台中\彰化\雲林\南投美食
台南\高雄\屏東美食
素食蔬果
蔬菜水果
五穀雜糧
素食料理
素食蔬果品牌總覽
調味醬料
醬料
調味品
料理油
調味醬料品牌總覽
490免運區
 蔬菜水果
五穀雜糧
肉乾滷味
米點\米食類
調味品
醬料
沖泡飲品\果汁
罐頭
料理油
休閒零食
調理即食
[INFO] famicloud_crawler complete!
OrderedDict([('food_info', OrderedDict([('food_name', '温國智古早味。彩虹銀絲卷'), ('original_price', '原價$200'), ('discount_price', '180'), ('stored_method', '冷凍'), ('category', ['限時好康', '限量9折', '限量搶購9折']), ('weight', '40g±5%/粒(8入/包)'), ('expired_date', '360天'), ('country_of_origin', '台灣')])), ('product_img_url', 'https://familycloud.blob.core.windows.net/products/10002208_0.jpg'), ('product_characteristic_img_url', 'https://familycloud.blob.core.windows.net/products/4500/2208-2NEW.jpg

IndexError: list index out of range

# 上層爬蟲

In [17]:
def famicloud_crawler(q):
    res_index = rq.get(index_url)
    soup_index = bs(res_index.text, 'lxml')
    soup_li = soup_index.li
    
    
    for i in range (1 ,1000):
        try:
            small_branch = soup_li.select('a')[i].text
            pre_small_branch_url = soup_li.select('a')[i]['href']
            small_branch_url = "{}{}".format(index_url, pre_small_branch_url)
            
            #print(small_branch_url)
            #交給中層爬蟲
            #pass
            #small_branch_crawler(small_branch_url)
            q.put(small_branch_url)
            

        except KeyError:       
            sub_branch = soup_li.select('a')[i].text
            print(sub_branch)

        except IndexError:
            break   

    print("[INFO] famicloud_crawler complete!")

# 中層爬蟲

In [9]:
def small_branch_crawler(small_branch_url):
    # 擷取 JavaScript 渲染的網頁原始碼，先指定 PhantomJs 安裝位置
    driver = webdriver.PhantomJS(executable_path='E:/phantomjs-2.1.1-windows/bin/phantomjs')
     # 輸入範例網址，交給瀏覽器 
    driver.get(small_branch_url)
    # 取得網頁原始碼 (含 JavaScript)
    pageSource = driver.page_source    
    # print(pageSource)
    soup = bs(pageSource, 'lxml')
    
    a_href_list = soup.select('#SmallMedium > a')
    for a_href in a_href_list:
        if a_href['href'] != 'javascript:void(0)':
            pre_product_url = a_href['href']
            product_url = "{}{}".format(HOST, pre_product_url)
            #print(product_url)
            product_page_crawler(product_url)
            

# 下層爬蟲

In [10]:
def product_page_crawler(product_url):
    res = rq.get(product_url)
    soup = bs(res.text, 'lxml')
    
    product_dict = od()
    food_info_dict = od()  
    
    food_name = ''
    
    #找到最後一個有strong標籤的div，就會是內容
    div_index_needed = 0
    for div_index in range(1, 70):
        if hasattr(soup.select("div")[div_index].strong, 'text'):
            if div_index > div_index_needed:
                div_index_needed = div_index
            #print(div_index_needed)
    #print(div_index_needed)
    
    try:
        soup.select("div")[div_index_needed].strong.text
        content = soup.select("div")[div_index_needed]
        new_content = content
        
        while(new_content.strong != None):
            
            try:
                food_name += new_content.strong.text
                new_content.strong.extract()
                
            except AttributeError:
                break
                
        food_info_dict["food_name"] = food_name
        #print(food_info_dict)
        
    #------以下應該都沒用了-----
    except AttributeError:
        soup.select("div")[54].strong.text
        content = soup.select("div")[54]
        new_content = content
        
        while(new_content.strong != None):
            
            try:   
                food_name += new_content.strong.text
                new_content.strong.extract()
                
            except AttributeError:
                break
                
        food_info_dict["food_name"] = food_name
    #------以上應該都沒用了-----    
    
    except:
        print("[***ERROR***]can not find food_name!")
        
    while(True):
        
        try:
            new_content.br.replace_with(' ')
            
        except AttributeError:
            break
   
    food_info_and_other = new_content.text.strip().split() 
    
    #original_price
    food_info_dict["original_price"] = soup.select(".BOriginalPrice")[0].text
    #discount_price
    food_info_dict["discount_price"] = soup.select(".OnSalePrice > b")[0].text
    #stored_method
    food_info_dict["stored_method"] = soup.select(".BProductLabel")[0].img['alt']
    #category
    category_list = []
    a_hrefs = soup.select('#map-list')[0]
    branch_levels = a_hrefs.select('a')
    for branch in branch_levels:
        if(branch.text != '首頁'):
            category_list.append(branch.text)
    food_info_dict["category"] = category_list
    #weight
    food_info_dict["weight"] = food_info_and_other[1]
    #expired_date
    food_info_dict["expired_date"] = food_info_and_other[7]
    #country_of_origin
    food_info_dict["country_of_origin"] = food_info_and_other[5]
    
    #把food_info字典放入product_dict字典
    product_dict["food_info"] = food_info_dict
    
    #steps(未完成)
    #product_dict["steps"] = null
    
    #product_img_url
    product_dict["product_img_url"] = soup.select(".product_img_box")[0].img['src']
    #product_characteristic_img_url
    
    try:
        #不在P標籤內
        product_dict["product_characteristic_img_url"] = soup.select('div > img')[1]['src']
        #rint(product_dict)
        
    except IndexError:
        #在P標籤內
        product_dict["product_characteristic_img_url"] = soup.select("div > p > img")[0]['src']
    
    #web_url
    product_dict["web_url"] = product_url
    
    print(product_dict)

In [14]:
famicloud_crawler()

TypeError: famicloud_crawler() missing 1 required positional argument: 'q'